In [1]:
import imageio.v3 as iio
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
depth_image = iio.imread('002.png')

# print properties:
print(f"Image resolution: {depth_image.shape}")
print(f"Data type: {depth_image.dtype}")
print(f"Min value: {np.min(depth_image)}")
print(f"Max value: {np.max(depth_image)}")

Image resolution: (480, 640)
Data type: int32
Min value: 0
Max value: 661


In [3]:
# Depth camera parameters:
FX_DEPTH = 525.0
FY_DEPTH = 525.0
CX_DEPTH = 319.5
CY_DEPTH = 239.5

In [4]:
# compute point cloud:
pcd = []
height, width = depth_image.shape
for i in range(height):
    for j in range(width):
        z = depth_image[i][j]
        x = (j - CX_DEPTH) * z / FX_DEPTH
        y = (i - CY_DEPTH) * z / FY_DEPTH
        pcd.append([x, y, z])

In [5]:
pcd_o3d = o3d.geometry.PointCloud()  # create point cloud object
pcd_o3d.points = o3d.utility.Vector3dVector(pcd)  # set pcd_np as the point cloud points
# Visualize:
o3d.visualization.draw_geometries([pcd_o3d])

In [8]:
print("Recompute the normal of the point cloud")
pcd_o3d.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1,
                                                          max_nn=30))
o3d.visualization.draw_geometries([pcd_o3d])

Recompute the normal of the downsampled point cloud


In [10]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud],
                                      zoom=0.3412,
                                      front=[0.4257, -0.2125, -0.8795],
                                      lookat=[2.6172, 2.0475, 1.532],
                                      up=[-0.0694, -0.9768, 0.2024])

In [17]:
print("Statistical oulier removal")
cl, ind = pcd_o3d.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=9.0)
display_inlier_outlier(pcd_o3d, ind)

Statistical oulier removal
Showing outliers (red) and inliers (gray): 


In [20]:
o3d.visualization.draw_geometries([cl])

In [21]:
distances = cl.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist

In [22]:
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(cl,o3d.utility.DoubleVector([radius, radius * 2]))

In [23]:
bpa_mesh.remove_degenerate_triangles()
bpa_mesh.remove_duplicated_triangles()
bpa_mesh.remove_duplicated_vertices()
bpa_mesh.remove_non_manifold_edges()

TriangleMesh with 21970 points and 22251 triangles.

In [31]:
o3d.visualization.draw_geometries([bpa_mesh])

In [25]:
type(bpa_mesh)

open3d.cpu.pybind.geometry.TriangleMesh

In [29]:
import skeletor as sk
import trimesh

tri_mesh = trimesh.Trimesh(np.asarray(bpa_mesh.vertices), np.asarray(bpa_mesh.triangles),
                          vertex_normals=np.asarray(bpa_mesh.vertex_normals))
fixed = sk.pre.fix_mesh(tri_mesh, remove_disconnected=5, inplace=False)
skel = sk.skeletonize.by_wavefront(fixed, waves=1, step_size=1)
skel

Skeletonizing:   0%|          | 0/11750 [00:00<?, ?it/s]

<Skeleton(vertices=(1142, 3), edges=(1134, 2), method=wavefront)>

In [30]:
skel.show(mesh=True)